#### Steps in this notebook
- Clean Cricsheet data, create basic model
- Run basic model for testing

In [141]:
import pandas as pd
import numpy as np
import math

In [16]:
cricsheet_df=pd.read_csv('./data/all_matches.csv')

/Users/kishan/Desktop/IPL/.ipl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
###Load input format dataframe

test_df=pd.read_csv('./submission/inputFile.csv')

test_df

##Venue needs to go through standardizer
## innigns is either 1 or 2
## Batting team and bowling team better to go through standardizer, since KXIP has changed names between seasons
## Batsmen list must be minimum 2 and bowlers list must be minimum 2
## Need to standardize batter and bowlers
## Batting and bowling types of batters and bowlers to be inferred from custom module

,venue,innings,batting_team,bowling_team,batsmen,bowlers
0,MA Chidambaram Stadium,1,Mumbai Indians,Kolkata Knight Riders,"Quinton de Kock, Rohit Sharma, Suryakumar Yadav","Harbhajan Singh, Varun Chakravarthy, Shakib Al..."


#### Subsetting the given data

- Remove seasons before 2018
- Remove super over innings (innings>=3)
- Remove unnecessary columns

In [17]:
##no use of other wicket type and other player dismissed columns
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_wicket_type'])].shape[0]==0
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_player_dismissed'])].shape[0]==0

In [28]:
cricsheet_df.drop(columns=['other_wicket_type','other_player_dismissed'],axis=1,inplace=True)

In [29]:
cricsheet_df['start_date']=pd.to_datetime(cricsheet_df['start_date'])

In [30]:
##Filter from 2018 April 7th
cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')

<ipython-input-30-ff7ecb1db089>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')


count                   43894
unique                    149
top       2020-10-18 00:00:00
freq                      520
first     2018-04-07 00:00:00
last      2021-04-11 00:00:00
Name: start_date, dtype: object

In [124]:
cdf=cricsheet_df.loc[cricsheet_df['start_date']>='2018'].reset_index(drop=True)

### EDA

In [125]:
##List of all venues in this
set(cdf['venue'])

{'Arun Jaitley Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Eden Gardens',
 'Holkar Cricket Stadium',
 'M.Chinnaswamy Stadium',
 'MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai',
 'Maharashtra Cricket Association Stadium',
 'Punjab Cricket Association IS Bindra Stadium',
 'Rajiv Gandhi International Stadium',
 'Sawai Mansingh Stadium',
 'Sharjah Cricket Stadium',
 'Sheikh Zayed Stadium',
 'Wankhede Stadium',
 'Wankhede Stadium, Mumbai'}

#### There are 231 different IPL players across teams from 2018. 

In [126]:
##List of all unique players in the dataset
ipl_players=cdf.loc[:,['striker','non_striker','bowler']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

len(ipl_players),ipl_players

(231,
 ['V Kohli',
  'PA Patel',
  'MM Ali',
  'AB de Villiers',
  'SO Hetmyer',
  'S Dube',
  'C de Grandhomme',
  'NA Saini',
  'YS Chahal',
  'UT Yadav',
  'Mohammed Siraj',
  'SR Watson',
  'AT Rayudu',
  'SK Raina',
  'KM Jadhav',
  'RA Jadeja',
  'DA Warner',
  'JM Bairstow',
  'V Shankar',
  'YK Pathan',
  'MK Pandey',
  'CA Lynn',
  'N Rana',
  'RV Uthappa',
  'KD Karthik',
  'AD Russell',
  'Shubman Gill',
  'PP Shaw',
  'S Dhawan',
  'SS Iyer',
  'CA Ingram',
  'RR Pant',
  'KMA Paul',
  'AR Patel',
  'R Tewatia',
  'RG Sharma',
  'Q de Kock',
  'SA Yadav',
  'KA Pollard',
  'Yuvraj Singh',
  'HH Pandya',
  'KH Pandya',
  'BCJ Cutting',
  'MJ McClenaghan',
  'Rasikh Salam',
  'KL Rahul',
  'MA Agarwal',
  'CH Gayle',
  'SN Khan',
  'N Pooran',
  'Mandeep Singh',
  'AM Rahane',
  'JC Buttler',
  'SV Samson',
  'SPD Smith',
  'RA Tripathi',
  'BA Stokes',
  'K Gowtham',
  'JC Archer',
  'JD Unadkat',
  'S Gopal',
  'DS Kulkarni',
  'MS Dhoni',
  'DJ Bravo',
  'SP Narine',
  'DA

In [127]:
### Format seasons
print(set(cdf['season']))

cdf['season']=cdf['season'].astype('str')
print(set(cdf['season']))

{2018, 2019, '2019', '2018', '2020/21', '2021'}
{'2020/21', '2018', '2019', '2021'}


In [128]:
##Remove super overs and repeated super over instance from MI KXIP
cdf=cdf.loc[cdf['innings']<=2]

In [129]:
##Which venues need tweaking? For purposes of training
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Sheikh Zayed Stadium                                   5294
Wankhede Stadium                                       3902
Eden Gardens                                           3802
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
Sawai Mansingh Stadium                                 3286
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
Punjab Cricket Association IS Bindra Stadium           2429
MA Chidambaram Stadium                                 2130
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
MA Chidambaram Stadium, Chepauk, Chennai                498
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Wankhede Stadium, Mumbai                                239
Name: venue, dtype: int64

In [130]:
cdf.loc[cdf['venue']=='MA Chidambaram Stadium, Chepauk, Chennai','venue']='MA Chidambaram Stadium'
cdf.loc[cdf['venue']=='Wankhede Stadium, Mumbai','venue']='Wankhede Stadium'

In [131]:
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Sheikh Zayed Stadium                                   5294
Wankhede Stadium                                       4141
Eden Gardens                                           3802
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
Sawai Mansingh Stadium                                 3286
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
MA Chidambaram Stadium                                 2628
Punjab Cricket Association IS Bindra Stadium           2429
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Name: venue, dtype: int64

In [132]:
##List of all unique teams in the dataset
ipl_teams=cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()
ipl_teams
##Change Delhi Daredevils to Delhi Capitals
##Change Kings XI Punjab to Punjab Kings

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Kings XI Punjab',
 'Rajasthan Royals',
 'Delhi Daredevils']

In [133]:
cdf.loc[cdf['batting_team']=='Delhi Daredevils','batting_team']='Delhi Capitals'
cdf.loc[cdf['bowling_team']=='Delhi Daredevils','bowling_team']='Delhi Capitals'
cdf.loc[cdf['batting_team']=='Kings XI Punjab','batting_team']='Punjab Kings'
cdf.loc[cdf['bowling_team']=='Kings XI Punjab','bowling_team']='Punjab Kings'

In [134]:
##Changed teams list

cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals']

In [137]:
##Extras column has captured type of extras
##At test time we will not have ball by ball information, do we need to hold a split of extras in that case??
cdf.drop(columns=['wides','noballs','byes','legbyes','penalty'],inplace=True)

### Derive new features

In [139]:
cdf['total_runs']=cdf['runs_off_bat']+cdf['extras']

In [142]:
##Get over number
cdf['over_number']=cdf['ball'].apply(lambda x:int(math.ceil(x)))

In [143]:
def phase_cat(over_number):
    '''Determine phase'''
    if over_number<=6:
        phase="PowerPlay"
    if (over_number>6) & (over_number<=15):
        phase="Middle overs"
    if over_number>15:
        phase="Death overs"
    return phase

In [144]:
cdf['phase']=cdf['over_number'].apply(phase_cat)

In [146]:
cdf.to_csv('./data/cleaned_cricsheet.csv',index=False)